<a href="https://colab.research.google.com/github/REICHIYAN/REI/blob/master/Nara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install icrawler

In [3]:
from icrawler.builtin import BingImageCrawler

crawler = BingImageCrawler(storage={"root_dir": "shika"})
crawler.crawl(keyword="しかまろくん", max_num=200)

2020-05-20 15:02:17,478 - INFO - icrawler.crawler - start crawling...
2020-05-20 15:02:17,479 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-05-20 15:02:17,483 - INFO - icrawler.crawler - starting 1 parser threads...
2020-05-20 15:02:17,485 - INFO - icrawler.crawler - starting 1 downloader threads...
2020-05-20 15:02:17,919 - INFO - parser - parsing result page https://www.bing.com/images/async?q=しかまろくん&first=0
2020-05-20 15:02:18,423 - INFO - downloader - image #1	https://narashikanko.or.jp/wp-content/themes/nara-portal/images/shikamaro/shikamaro.jpg
2020-05-20 15:02:18,687 - INFO - downloader - image #2	http://stat.ameba.jp/user_images/20140927/00/sandy-emilia/b0/52/j/o0396040013079359278.jpg
2020-05-20 15:02:20,126 - INFO - downloader - image #3	https://stat.ameba.jp/user_images/20190103/21/yasumilu/30/6f/j/o0810108014332705141.jpg
2020-05-20 15:02:20,739 - INFO - downloader - image #4	https://hbb.afl.rakuten.co.jp/hgb/?pc=http:%2f%2fthumbnail.image.rakuten.co.jp%2f%4

In [4]:
from icrawler.builtin import BingImageCrawler

crawler = BingImageCrawler(storage={"root_dir": "sento"})
crawler.crawl(keyword="せんとくん", max_num=200)

2020-05-20 15:03:58,106 - INFO - icrawler.crawler - start crawling...
2020-05-20 15:03:58,109 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-05-20 15:03:58,110 - INFO - icrawler.crawler - starting 1 parser threads...
2020-05-20 15:03:58,116 - INFO - icrawler.crawler - starting 1 downloader threads...
2020-05-20 15:03:58,524 - INFO - parser - parsing result page https://www.bing.com/images/async?q=せんとくん&first=0
2020-05-20 15:03:58,831 - INFO - downloader - image #1	https://image.itmedia.co.jp/news/articles/0806/02/yuo_manto_02.jpg
2020-05-20 15:03:59,680 - INFO - downloader - image #2	https://blog-imgs-24-origin.fc2.com/k/n/i/knightsendnight/20080416204616.jpg
2020-05-20 15:04:00,303 - INFO - downloader - image #3	https://sites.google.com/site/jpdanara/_/rsrc/1395929836256/home/%E3%81%9B%E3%82%93%E3%81%A8%E3%81%8F%E3%82%93.jpg
2020-05-20 15:04:00,788 - INFO - downloader - image #4	http://blog-imgs-23-origin.fc2.com/g/o/r/gorockskate/trd0804151658021-p4.jpg
2020-05-20 15:0

In [0]:
from PIL import Image
import os, glob
import numpy as np
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

classes = ["sento", "shika"]
num_classes = len(classes)
image_size = 64
num_testdata = 25

X_train = []
X_test  = []
y_train = []
y_test  = []

for index, classlabel in enumerate(classes):
    photos_dir = "./" + classlabel
    files = glob.glob(photos_dir + "/*.jpg")
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        data = np.asarray(image)
        if i < num_testdata:
            X_test.append(data)
            y_test.append(index)
        else:
            for angle in range(-20, 20, 5):

                img_r = image.rotate(angle)
                data = np.asarray(img_r)
                X_train.append(data)
                y_train.append(index)
                
                img_trans = img_r.transpose(Image.FLIP_LEFT_RIGHT)
                data = np.asarray(img_trans)
                X_train.append(data)
                y_train.append(index)

X_train = np.array(X_train)
X_test  = np.array(X_test)
y_train = np.array(y_train)
y_test  = np.array(y_test)

xy = (X_train, X_test, y_train, y_test)
np.save("./sento_shika.npy", xy)

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import keras
import numpy as np

classes = ["sento", "shika"]
num_classes = len(classes)
image_size = 64

def load_data():
    X_train, X_test, y_train, y_test = np.load("./sento_shika.npy", allow_pickle=True)

    X_train = X_train.astype("float") / 255
    X_test  = X_test.astype("float") / 255

    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test  = np_utils.to_categorical(y_test, num_classes)

    return X_train, y_train, X_test, y_test


def train(X, y, X_test, y_test):
    model = Sequential()

    model.add(Conv2D(32,(3,3), padding='same',input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(64,(3,3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64,(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.45))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = keras.optimizers.rmsprop(lr=0.00005, decay=1e-6)

    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    model.fit(X, y, batch_size=28, epochs=40)

    model.save('./cnn.h5')

    return model


def main():

    X_train, y_train, X_test, y_test = load_data()

    model = train(X_train, y_train, X_test, y_test)

main()

Using TensorFlow backend.


Epoch 1/40
3024/3024 [==============================] - 9s 3ms/step - loss: 0.6295 - accuracy: 0.6478
Epoch 2/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.5234 - accuracy: 0.7411
Epoch 3/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.4620 - accuracy: 0.7877
Epoch 4/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.4109 - accuracy: 0.8155
Epoch 5/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.3720 - accuracy: 0.8446
Epoch 6/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.3315 - accuracy: 0.8614
Epoch 7/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.2934 - accuracy: 0.8886
Epoch 8/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.2600 - accuracy: 0.8952
Epoch 9/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.2301 - accuracy: 0.9107
Epoch 10/40
3024/3024 [==============================] - 3s 1ms/step - loss: 0.206

In [10]:
import keras
import sys, os
import numpy as np
from keras.models import load_model

imsize = (64, 64)


testpic     = "./shika1.jpg"
keras_param = "./cnn.h5"

def load_image(path):
    img = Image.open(path)
    img = img.convert('RGB')

    img = img.resize(imsize)

    img = np.asarray(img)
    img = img / 255.0
    return img

if __name__ == "__main__":

    model = load_model(keras_param)
    img = load_image(testpic)
    prd = model.predict(np.array([img]))
    print(prd)
    prelabel = np.argmax(prd, axis=1)
    if prelabel == 0:
        print(">>> せんとくん")
    elif prelabel == 1:
        print(">>> しかまろくん")

[[1.4627471e-14 1.0000000e+00]]
>>> しかまろくん
